In [1]:
# Importo librerías necesarias

import cv2
import pylab as plt
import matplotlib
import numpy as np
import pickle
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
import time
import pykalman
from scipy.optimize import curve_fit
import functools
import traceback
from scipy.ndimage.filters import gaussian_filter

(cv2version, _, _) = cv2.__version__.split(".")
cv2version = int(cv2version)

In [11]:
#main_dir = "C:/Users/Labo/Google Drive/Multimodal Integration/Scripts/Tracking/results/temp"
#main_dir = "C:/Users/Labo/Google Drive/Multimodal Integration/Scripts/Tracking/results/MULTI3"
# main_dir = "C:/Users/PC/My Drive/Multimodal Integration/Scripts/Tracking/results/MULTI2"
main_dir = 'C:/Users/PC/My Drive/Multimodal Integration/Scripts/Tracking/results/MULTI1/Muestreo'
#main_dir = "D:/Videos Zebrafish"
os.chdir(main_dir)

video_ids = []
datasets = []
max_datasets = 50
count = 0

for d in os.listdir():
    if ".xlsx" in d:
        print("Reading: " + str(d))
        video_ids.append(d)
        datasets.append(pd.read_excel(d))
        count = count + 1
    if count >= max_datasets:
        break

Reading: multi1-11012021153455-0000.xlsx
Reading: multi1-11012021153607-0000.xlsx
Reading: multi1-11012021154305-0000.xlsx
Reading: multi1-11022021101732-0000.xlsx
Reading: multi1-11022021102521-0000.xlsx
Reading: multi1-11022021102724-0000.xlsx
Reading: multi1-11022021104027-0000.xlsx
Reading: multi1-11022021104746-0000.xlsx
Reading: multi1-11022021110457-0000.xlsx
Reading: multi1-11022021112327-0000.xlsx
Reading: multi1-11022021113249-0000.xlsx
Reading: multi1-11022021114745-0000.xlsx
Reading: multi1-11022021120300-0000.xlsx
Reading: multi1-11022021121422-0000.xlsx
Reading: multi1-11022021123056-0000.xlsx
Reading: multi1-11022021124146-0000.xlsx
Reading: multi1-11022021130227-0000.xlsx
Reading: multi1-11022021130853-0000.xlsx
Reading: multi1-11022021132657-0000.xlsx
Reading: multi1-11022021133325-0000.xlsx
Reading: multi1-11022021135613-0000.xlsx
Reading: multi1-11022021143701-0000.xlsx
Reading: multi1-11022021144437-0000.xlsx
Reading: multi1-11022021145328-0000.xlsx
Reading: multi1-

In [12]:
n = 4

# promedio un array en ventanas de n elementos (moving window con stride de 1)
def moving_average(arr, n):
    arr = np.array(arr)
    
    final_n = len(arr)-n + 1
    
    res = np.zeros((final_n,))
    
    for i in range(final_n):
        res[i] = arr[i:i+n].mean(axis=0)
    
    return res

# promedio un array cada n elementos
def average_by(arr, n):
    arr = np.array(arr)
    
    modn = len(arr)%n
    
    if modn != 0:
        arr = arr[:-modn]
    
    res = np.mean(arr.reshape(-1, n), axis=1)
    
    return res

# toma un array de posiciones (x, y) y lo transforma a un array de distancias entre cada posicion consecutiva
def pos_to_dist(pos, n = None, avg = moving_average):
    pos = np.array(pos)
    
    if n is not None:
        posx = avg(pos[:,0], n)
        posy = avg(pos[:,0], n)
        pos = np.column_stack([posx, posy])
    
    deltas = np.diff(pos, axis = 0)
    dx = deltas[:,0]
    dy = deltas[:,1]
    dist = np.sqrt(dx**2 + dy**2)
    return dist

def get_plot_data(frames, ys, n, width = 600, height = 200):
    
    #ys = moving_average(ys,n)
    #frames = moving_average(frames,n)
    
    ys = gaussian_filter(ys, sigma=n)
    frames = gaussian_filter(frames, sigma=n)
    
    ys = np.array(ys)
    frames = np.array(frames)
    
    miny = ys.min()
    maxy = ys.max()
    
    minf = frames.min()
    maxf = frames.max()
    
    initial_ys = []
    final_ys = []
    
    initial_xs = []
    final_xs = []
    
    final_frames = []
    
    for i in range(len(ys)-1):
        yi = ys[i]
        yf = ys[i+1]
        fi = frames[i]
        ff = frames[i+1]
        
        xi = int(np.interp(fi, [minf, maxf], [0, width]))
        xf = int(np.interp(ff, [minf, maxf], [0, width]))
        
        yi = int(np.interp(yi, [miny, maxy], [height, 0]))
        yf = int(np.interp(yf, [miny, maxy], [height, 0]))
        
        initial_xs.append(xi)
        initial_ys.append(yi)
        final_xs.append(xf)
        final_ys.append(yf)
        final_frames.append(ff)
    
    
    return initial_xs, initial_ys, final_xs, final_ys, final_frames, width, height

def plot_data(data, frame, name):
    initial_xs, initial_ys, final_xs, final_ys, final_frames, width, height = data
    
    window = np.zeros((height, width, 3), np.uint8)
    
    for i in range(len(initial_ys)-1):
        if final_frames[i] < frame:
            color = (0, 255, 255)
        else:
            color = (255, 255, 0)
        
        cv2.line(window, (initial_xs[i], initial_ys[i]), (final_xs[i], final_ys[i]), color = color, thickness = 1)
    
    cv2.imshow(name, window)

def show_tracking_data(video_dir, data):
    vidcap = cv2.VideoCapture(video_dir)
    
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    success,image = vidcap.read()

    print("Opened video: " + str(success))
    
    if not success:
        return
    
    broken = False
    step = 25
    zoom = 1.1
    
    video_name = video_dir.split('/')[-1]
    
    xs = []
    ys = []

    for (columnName, columnData) in data.iteritems():
        if "axis_point" in columnName:
            index = columnName.split("_")[2]
            var = columnName.split("_")[3]

            values = []

            for row in range(len(data)):
                values.append(data[columnName][row])

            if var == "x":
                xs.append(values)
            elif var == "y":
                ys.append(values)

    axis = []

    for index in range(len(xs)):
        x = np.array(xs[index])
        y = np.array(ys[index])

        points = np.vstack((x,y)).T

        axis.append(points)

    axis = np.stack(axis, axis=1)
    
    positions = []
    startPos = 0
    
    play = False
    prev = False
    show = True
    wholeT = True
    exclude = False
    
    for i in range(len(data)):
        pos = [data["anterior_x"][i],data["anterior_y"][i]]
        positions.append(pos)
    
    ys = []
    frames = []
    
    for i in range(len(data)):
        ys.append(data["anterior_angular_change"][i])
        frames.append(data["frame"][i])
    
    ang_data = get_plot_data(frames, ys, n)
    
    ys = pos_to_dist(positions, n = n, avg = average_by)
    frames = average_by(frames, n)
    vel_data = get_plot_data(frames, ys, n)
    
    acc = np.diff(ys)
    frames = frames[:-1]
    acc_data = get_plot_data(frames, acc, n)
    
    i = 0
    while i < len(data) and not broken:
        if i < 0:
            i = 0

        if i >= len(data):
            break
        
        frame = data["frame"][i]
        pos = positions[i]
        
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)
        success, image = vidcap.read()

        width = image.shape[1]
        height = image.shape[0]
        
        cv2.resizeWindow('image',int(width*zoom),int(height*zoom))
        
        # show velocities
        plot_data(vel_data, frame, "velocity")
        plot_data(ang_data, frame, "angular velocity")
        plot_data(acc_data, frame, "acceleration")
        
        # draw trajectory
        
        if show:
            
            endPos = len(data)-2 if wholeT else i-2
            for j in range(startPos, endPos):
                if j < 0:
                    continue

                pos_i = positions[j]
                pos_f = positions[j+1]

                if pos_i is None or pos_f is None:
                    continue

                pos_i = [int(pos_i[0]), int(pos_i[1])]
                pos_f = [int(pos_f[0]), int(pos_f[1])]

                cv2.line(image, tuple(pos_i), tuple(pos_f), color = (0,0,0), thickness = 1)

            # draw centers
            for j in range(axis.shape[1]):
                center = axis[i, j, :]
                cv2.circle(image, (int(center[0]), int(center[1])), 1, (255,0,150), -1)

            # draw lines
            for j in range(axis.shape[1] - 1):
                p1 = axis[i, j, :]
                p2 = axis[i, j+1, :]

                p1 = (int(p1[0]), int(p1[1]))
                p2 = (int(p2[0]), int(p2[1]))

                cv2.line(image, p1, p2, color = (0, 255, 0), thickness = 1)

            # draw anterior
            cv2.circle(image, (int(pos[0]), int(pos[1])), 2, (0,0,255), -1)
        
        # frame number
        cv2.putText(image, "Frame: " + str(frame),
                    (10,25), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,(0,0,255), 2)
        # frame number
        cv2.putText(image, "Step: " + str(step),
                    (10,50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,(0,0,255), 2)
        # frame number
        cv2.putText(image, "Video: " + str(video_name),
                    (10,height-25), cv2.FONT_HERSHEY_SIMPLEX,
                    0.45,(0,0,255), 1)
        # frame number
        cv2.putText(image, "Exclude: " + str(exclude),
                    (width - 110,height-25), cv2.FONT_HERSHEY_SIMPLEX,
                    0.45,(0,0,255), 1)
        
        # show image
        cv2.imshow('image',image)
        
        k = cv2.waitKey(1) & 0xFF

        if k == 27:
            broken = True
        elif k == ord('y') or k == ord('Y'):       # si pulsé 'f', avanzo N fotogramas en el video
            show = not show
        elif k == ord('f') or k == ord('F'):       # si pulsé 'f', avanzo N fotogramas en el video
            i = i + 200
        elif k == ord('d') or k == ord('D'):       # si pulsé 'f', avanzo N fotogramas en el video
            i = i - 200
        elif k == ord('p') or k == ord('P'):
            play = not play
            wholeT = not wholeT
        elif k == ord('o') or k == ord('O'):
            step = step + 1
        elif k == ord('i') or k == ord('I'):
            step = max(step - 1, 1)
        elif k == ord('x') or k == ord('X'):
            exclude = not exclude
        elif k == 54:
            i = i + 1
        elif k == 53:
            i = max(i - 1, 0)
        elif k == 50:
            zoom = zoom + 0.05
        elif k == 49:
            zoom = max(zoom - 0.05, 0.1)
        elif k == ord('n') or k == ord('N'):
            break
        elif k == ord('b') or k == ord('B'):
            prev = True
            break
        
        if play:
            i = i + step
    
    return broken, prev, exclude

In [13]:
#datasets["dincal-09062021112824-0000.xlsx"]["frame"][0]

video_to_watch = None
#video_to_watch = "prueba-02222022151449-0000"

video_dir = "D:/Videos Zebrafish"
#video_dir = "D:/Videos Zebrafish/Multi3 (MULTI3 - Enero 2022)"

os.chdir(video_dir)

directories = []

def look_for_video(main):
    for d in os.listdir():
        os.chdir(main)
        v = main + '/' + d
        if '.' in v:
            directories.append(v)
        else:
            os.chdir(v)
            look_for_video(v)

look_for_video(os.getcwd())

prev = False
excluded = []
z = 0

cv2.namedWindow('image', cv2.WINDOW_NORMAL)

cv2.setWindowProperty('image',cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
cv2.setWindowProperty('image',cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_NORMAL)

while z < len(video_ids):
    print(z)
    video_id = video_ids[z].split('.')[0]
    print(video_id)
    
    if video_to_watch is not None and video_to_watch not in video_id:
        print(video_to_watch)
        print(video_id)
        z = z + 1
        continue
    
    dataset = datasets[z]
    
    d = None
    for directory in directories:
        if video_id in directory:
            d = directory
            print(d)
            break
    if d is not None:
        print(d)
        
        try:
            broken, prev, exclude = show_tracking_data(d, dataset)
        except:
            cv2.destroyAllWindows()
            traceback.print_exc()
            break
        
        if exclude:
            excluded.append(video_id)
        if broken:
            break
        if prev:
            print("Going back")
            z = max(z - 1, 0)
        else:
            z = z + 1
    
    
cv2.destroyAllWindows()
print("Excluded videos:")
print(excluded)

0
multi1-11012021153455-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-01/8/multi1-11012021153455-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-01/8/multi1-11012021153455-0000.avi
Opened video: True
1
multi1-11012021153607-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-01/8/multi1-11012021153607-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-01/8/multi1-11012021153607-0000.avi
Opened video: True
2
multi1-11012021154305-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-01/8/multi1-11012021154305-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-01/8/multi1-11012021154305-0000.avi
Opened video: True
3
multi1-11022021101732-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-02/1/multi1-11022021101732-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-02/1/multi1-11022021101732-0000.avi
Opened video: True
4
multi1-11022021102521-0000
D:\

30
multi1-11042021114057-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/4/multi1-11042021114057-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/4/multi1-11042021114057-0000.avi
Opened video: True
31
multi1-11042021115344-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/4/multi1-11042021115344-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/4/multi1-11042021115344-0000.avi
Opened video: True
32
multi1-11042021121316-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/5/multi1-11042021121316-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/5/multi1-11042021121316-0000.avi
Opened video: True
33
multi1-11042021123814-0000
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/6/multi1-11042021123814-0000.avi
D:\Videos Zebrafish/Multi1 (MULTI1 - Noviembre 2021)/2021-11-04/6/multi1-11042021123814-0000.avi
Opened video: True
34
multi1-11042021124944-000

In [6]:
len(excluded)/len(datasets)

0.0

Excluded videos:
'alcohol3-03282022121500-0000', 'alcohol3-03282022171701-0000', 'alcohol3-03302022111920-0000', 'alcohol3-03302022144412-0000', 'alcohol3-03302022145301-0000', 'multi1-11012021120136-0000', 'multi1-11012021130713-0000', 'multi1-11022021104746-0000', 'multi1-11022021110457-0000'